In [1]:
import numpy as np
import pandas as pd

# to access public datasets from the knoema.com
import knoema

import bamboolib

# 0. Theoretical background

$$r_t^{eqy} = E[r_t^{eqy}] + \beta_1^{eqy} INFL_t+\beta_2^{eqy} GR_t+ \beta_3^{eqy} FS_t + \alpha_t^{eqy}$$
$$r_t^{ust} = E[r_t^{ust}] + \beta_1^{ust} INFL_t+\beta_2^{ust} GR_t+ \beta_3^{ust} FS_t + \alpha_t^{ust}$$
$$\vdots$$
$$r_t^{fxcs} = E[r_t^{fxcs}] + \beta_1^{fxcs} INFL_t+\beta_2^{fxcs} GR_t+ \beta_3^{fxcs} FS_t + \alpha_t^{fxcs}$$

, where
- $INFL_t$ stands for an *inflation* macro-factor return at time t. Likewise, $GR$ for *growth* and $FS$ for *finantial stress* factors.
- $r_t^{eqy}$ is an *excess* return of global equity markets at time t as one of the base assets. T-Bill 1M is used for the excess return calculation. A risk free return such as $r_f$ is omiited for simplicity.
  - $ust$ stands for U.S 10Yr Treasury, and the rest of notation should be the same as that of $eqy$. The same naming rule applies for the rest of base assets in denoting $r_t^{asset}$.
  - A full list of base assets used in this model : Equities($eqy$), Treasuries($ust$), Credit($cre$), Inflation-Linked Bonds($ilb$), Gold(gold), Industrial Metals($inm$), Energy commodity($eng$), U.S. Dollar($dxy$), Commodity vs safe haven currencies ($fxcs$). Abbreviations in ().
- $E[\cdot]$ is an expected excess return.
- $\beta_{\#}^{instrument}$ is a factor beta, or factor loading, for that $instrument$. This value is the **same across all periods** of time being modeled. Therefore, there is no subscript $t$. Instead, we have a digit subscript 1 for $INFL$ beta, 2 for $GR$ beta and 3 for $FS$ beta.
  - e.g. $\beta_1^{eqy}$ is a sensitivy of Equities to $INFL$ factor.
- $E[\alpha^{asset}] = 0$ for all periods of time being modeled.

# 1. Load datasets

#### From Bloomberg
- For data of more recent years.

In [2]:
bbg_filename = '../../data/raw/base_assets_price_bbg_M.log'

df_bbg = pd.read_csv(bbg_filename, header=2, parse_dates=['date'])
df_bbg = df_bbg.set_index(['date'], drop=True)

In [3]:
df_bbg.tail()

,DMEQ,GLT,CRE_Baa,CRE_Aaa,ILB,GOLD,INM,ENGY,DXY,USDCAD,USDNOK,AUDUSD,USDJPY,USDCHF,TBILL3M,EMEQ,ACEQ
date,,,,,,,,,,,,,,,,,
2019-11-29,2292.26,2383.16,4206.804,2246.016,316.2039,1463.98,314.60,190.0800,98.273,1.3282,9.2248,0.6763,109.49,1.0002,1.566108,1040.05,546.70
2019-12-31,2358.47,2369.78,4233.494,2244.171,317.4027,1517.27,324.21,206.6327,96.389,1.2990,8.7855,0.7021,108.61,0.9666,1.525863,1114.66,565.24
2020-01-31,2342.41,2427.69,4332.237,2281.815,324.0671,1589.16,301.55,174.8693,97.390,1.3237,9.2011,0.6692,108.35,0.9634,1.535940,1062.34,558.62
2020-02-28,2141.12,2492.04,4372.826,2320.946,328.5524,1585.69,298.11,154.7120,98.132,1.3407,9.3939,0.6515,107.89,0.9649,1.510747,1005.52,512.76
2020-03-31,1852.73,2564.12,3920.704,2333.130,322.7761,1577.18,267.86,82.0629,99.048,1.4062,10.4028,0.6131,107.54,0.9611,0.085018,848.58,442.35


#### From Global Financial Data (GFD)
- For data of older years; since 1850.

In [4]:
gfd_filename = '../../data/raw/base_assets_price_gfd_M.log'

df_gfd = pd.read_csv(gfd_filename, header=0, parse_dates=['Date'])

In [6]:
df_gfd.head()

,Date,Ticker,Close
0,1859-09-30,__WTC_D,20.00
1,1859-10-31,__WTC_D,20.00
2,1859-11-30,__WTC_D,20.00
3,1859-12-31,__WTC_D,20.00
4,1860-01-31,__WTC_D,19.25


We pivot this table.

In [7]:
df_gfd = pd.pivot_table(df_gfd, index=['Date'], columns=['Ticker'])

# Drop one of the top multi-index column, namely, "Close"
df_gfd.columns = df_gfd.columns.droplevel(0)

# 2. Preprocessing

### Changing `date` formats to properly join

#### A different format is being used in each index
- YYYY-MM-**01** vs YYYY-MM-**TheEndOfMonth**

In [9]:
df_gfd.index

DatetimeIndex(['1850-01-31', '1850-02-28', '1850-03-31', '1850-04-30',
               '1850-05-31', '1850-06-30', '1850-07-31', '1850-08-31',
               '1850-09-30', '1850-10-31',
               ...
               '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
               '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29',
               '2020-03-31', '2020-04-30'],
              dtype='datetime64[ns]', name='Date', length=2044, freq=None)

In [10]:
df_bbg.index

DatetimeIndex(['1959-12-31', '1960-01-29', '1960-02-29', '1960-03-31',
               '1960-04-29', '1960-05-31', '1960-06-30', '1960-07-29',
               '1960-08-31', '1960-09-30',
               ...
               '2019-06-28', '2019-07-31', '2019-08-30', '2019-09-30',
               '2019-10-31', '2019-11-29', '2019-12-31', '2020-01-31',
               '2020-02-28', '2020-03-31'],
              dtype='datetime64[ns]', name='date', length=724, freq=None)

#### Change each to `PeriodIndex` with **M** frequency

In [13]:
gfd_index = pd.to_datetime(df_gfd.index).to_period('M')
bbg_index = pd.to_datetime(df_bbg.index).to_period('M')

In [14]:
df_gfd = df_gfd.set_index(gfd_index)
df_bbg = df_bbg.set_index(bbg_index)

In [28]:
df_gfd.index

PeriodIndex(['1850-01', '1850-02', '1850-03', '1850-04', '1850-05', '1850-06',
             '1850-07', '1850-08', '1850-09', '1850-10',
             ...
             '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
             '2020-01', '2020-02', '2020-03', '2020-04'],
            dtype='period[M]', name='Date', length=2044, freq='M')

In [29]:
bbg_index

PeriodIndex(['1959-12', '1960-01', '1960-02', '1960-03', '1960-04', '1960-05',
             '1960-06', '1960-07', '1960-08', '1960-09',
             ...
             '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11',
             '2019-12', '2020-01', '2020-02', '2020-03'],
            dtype='period[M]', name='date', length=724, freq='M')

In [24]:
df_gfd.head()

Ticker,CHFUSD,CMWXMMM,CPUSAM,GBPUSD,IDUSAD,ITUSA3CMD,NG_P_WTD,TRUSACOM,TRUSG10M,USDBEF,...,USDFRF,USDITL,USDJPY,USDNLG,USDNOK,USDSEK,_DJCBPD,_SPXD,__WTC_D,__XAU_D
Date,,,,,,,,,,,,,,,,,,,,,
1850-01,0.053528,NaN,NaN,23.047528,NaN,NaN,NaN,1.051075,3.306525,5.2607,...,5.2938,5.4888,NaN,2.5038,3.8491,3.7465,NaN,NaN,NaN,20.6718
1850-02,0.053811,NaN,NaN,23.428364,NaN,NaN,NaN,1.055901,3.315689,5.2468,...,5.2798,5.3614,NaN,2.4845,3.8318,3.7535,NaN,NaN,NaN,20.6718
1850-03,0.053698,NaN,NaN,23.360561,NaN,NaN,NaN,1.052282,3.273533,5.2524,...,5.2854,5.3675,NaN,2.4845,3.8377,3.7621,NaN,NaN,NaN,20.6718
1850-04,0.053186,NaN,NaN,23.091783,NaN,NaN,NaN,1.057999,3.475150,5.2775,...,5.3107,5.4253,NaN,2.5237,3.8457,3.7699,NaN,NaN,NaN,20.6718
1850-05,0.039322,NaN,NaN,23.911980,NaN,NaN,NaN,1.081762,3.482482,5.2330,...,5.2659,5.3273,NaN,2.4845,3.7653,3.7356,NaN,NaN,NaN,20.6718


In [31]:
df_bbg.tail()

,DMEQ,GLT,CRE_Baa,CRE_Aaa,ILB,GOLD,INM,ENGY,DXY,USDCAD,USDNOK,AUDUSD,USDJPY,USDCHF,TBILL3M,EMEQ,ACEQ
date,,,,,,,,,,,,,,,,,
2019-11,2292.26,2383.16,4206.804,2246.016,316.2039,1463.98,314.60,190.0800,98.273,1.3282,9.2248,0.6763,109.49,1.0002,1.566108,1040.05,546.70
2019-12,2358.47,2369.78,4233.494,2244.171,317.4027,1517.27,324.21,206.6327,96.389,1.2990,8.7855,0.7021,108.61,0.9666,1.525863,1114.66,565.24
2020-01,2342.41,2427.69,4332.237,2281.815,324.0671,1589.16,301.55,174.8693,97.390,1.3237,9.2011,0.6692,108.35,0.9634,1.535940,1062.34,558.62
2020-02,2141.12,2492.04,4372.826,2320.946,328.5524,1585.69,298.11,154.7120,98.132,1.3407,9.3939,0.6515,107.89,0.9649,1.510747,1005.52,512.76
2020-03,1852.73,2564.12,3920.704,2333.130,322.7761,1577.18,267.86,82.0629,99.048,1.4062,10.4028,0.6131,107.54,0.9611,0.085018,848.58,442.35


In [ ]:
df_bbg

In [22]:
df_gfd
# bamboolib live code export
df_gfd = df_gfd.reset_index()
df_gfd

Ticker,CHFUSD,CMWXMMM,CPUSAM,GBPUSD,IDUSAD,ITUSA3CMD,NG_P_WTD,TRUSACOM,TRUSG10M,USDBEF,...,USDFRF,USDITL,USDJPY,USDNLG,USDNOK,USDSEK,_DJCBPD,_SPXD,__WTC_D,__XAU_D
Date,,,,,,,,,,,,,,,,,,,,,
1850-01,0.053528,NaN,NaN,23.047528,NaN,NaN,NaN,1.051075,3.306525,5.2607,...,5.2938,5.4888,NaN,2.5038,3.8491,3.7465,NaN,NaN,NaN,20.6718
1850-02,0.053811,NaN,NaN,23.428364,NaN,NaN,NaN,1.055901,3.315689,5.2468,...,5.2798,5.3614,NaN,2.4845,3.8318,3.7535,NaN,NaN,NaN,20.6718
1850-03,0.053698,NaN,NaN,23.360561,NaN,NaN,NaN,1.052282,3.273533,5.2524,...,5.2854,5.3675,NaN,2.4845,3.8377,3.7621,NaN,NaN,NaN,20.6718
1850-04,0.053186,NaN,NaN,23.091783,NaN,NaN,NaN,1.057999,3.475150,5.2775,...,5.3107,5.4253,NaN,2.5237,3.8457,3.7699,NaN,NaN,NaN,20.6718
1850-05,0.039322,NaN,NaN,23.911980,NaN,NaN,NaN,1.081762,3.482482,5.2330,...,5.2659,5.3273,NaN,2.4845,3.7653,3.7356,NaN,NaN,NaN,20.6718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12,1.067163,77.469997,256.974,1.759655,NaN,1.55,NaN,23920.300000,10836.125049,0.8917,...,0.8917,0.8917,108.61,0.8917,8.7784,9.3620,128.0591,3230.78,61.14,1517.0100
2020-01,1.077882,77.702232,257.971,1.742672,NaN,1.55,NaN,24418.540000,11252.031439,0.9013,...,0.9013,0.9013,108.38,0.9013,9.1991,9.6261,130.8381,3225.52,51.58,1589.8101
2020-02,1.072827,73.015869,258.678,1.643800,NaN,1.27,NaN,24727.670000,11666.405078,0.9069,...,0.9069,0.9069,108.07,0.9069,9.3945,9.5959,131.9078,2954.22,44.83,1584.7400


#### Joining `df_gfd` and `df_bbg` to create `df`

In [38]:
df = pd.merge(df_gfd, df_bbg, how='left', left_index=True, right_index=True)

Data range is:

In [39]:
df

,CHFUSD,CMWXMMM,CPUSAM,GBPUSD,IDUSAD,ITUSA3CMD,NG_P_WTD,TRUSACOM,TRUSG10M,USDBEF,...,ENGY,DXY,USDCAD_y,USDNOK_y,AUDUSD,USDJPY_y,USDCHF,TBILL3M,EMEQ,ACEQ
Date,,,,,,,,,,,,,,,,,,,,,
1850-01,0.053528,NaN,NaN,23.047528,NaN,NaN,NaN,1.051075,3.306525,5.2607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-02,0.053811,NaN,NaN,23.428364,NaN,NaN,NaN,1.055901,3.315689,5.2468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-03,0.053698,NaN,NaN,23.360561,NaN,NaN,NaN,1.052282,3.273533,5.2524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-04,0.053186,NaN,NaN,23.091783,NaN,NaN,NaN,1.057999,3.475150,5.2775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850-05,0.039322,NaN,NaN,23.911980,NaN,NaN,NaN,1.081762,3.482482,5.2330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12,1.067163,77.469997,256.974,1.759655,NaN,1.55,NaN,23920.300000,10836.125049,0.8917,...,206.6327,96.389,1.2990,8.7855,0.7021,108.61,0.9666,1.525863,1114.66,565.24
2020-01,1.077882,77.702232,257.971,1.742672,NaN,1.55,NaN,24418.540000,11252.031439,0.9013,...,174.8693,97.390,1.3237,9.2011,0.6692,108.35,0.9634,1.535940,1062.34,558.62
2020-02,1.072827,73.015869,258.678,1.643800,NaN,1.27,NaN,24727.670000,11666.405078,0.9069,...,154.7120,98.132,1.3407,9.3939,0.6515,107.89,0.9649,1.510747,1005.52,512.76


In [40]:
df.index.min(), df.index.max()

(Period('1850-01', 'M'), Period('2020-04', 'M'))

# 3. Generate indices
- `CRE`: U.S Credit Baa index vs Aaa index
- `ILB`: A proxy of TIPS for old days from January 1997 backwords. Following Swinkels (2018), it's spread return based on estimated real yields changes.

In [15]:
df
# bamboolib live code export
df = df.reset_index()
df

,WEQ,GLT,CRE_Baa,CRE_Aaa,ILB,GOLD,INM,ENGY,DXY,CADUSD,...,CHFUSD,LIBOR1M,FED_RATE,FED_EFFRATE,aluminum,copper,lead,zinc,crude,natural_gas
date,,,,,,,,,,,,,,,,,,,,,
1960-01,NaN,NaN,NaN,NaN,NaN,35.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.99,511.471832,715.40,206.10,260.80,1.63,0.1400
1960-02,NaN,NaN,NaN,NaN,NaN,35.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.97,511.471832,728.19,203.70,244.90,1.63,0.1400
1960-03,NaN,NaN,NaN,NaN,NaN,35.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.84,511.471832,684.94,210.30,248.70,1.63,0.1400
1960-04,NaN,NaN,NaN,NaN,NaN,35.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.92,511.471832,723.11,213.60,254.60,1.63,0.1400
1960-05,NaN,NaN,NaN,NaN,NaN,35.27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.85,511.471832,684.75,213.40,253.80,1.63,0.1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11,2292.26,2383.16,4206.804,2246.016,316.2039,1463.98,314.60,190.0800,98.273,0.7530,...,0.9999,1.69713,1.75,1.55,1774.790000,5859.95,2021.15,2425.48,61.41,2.6505
2019-12,2358.47,2369.78,4233.494,2244.171,317.4027,1517.27,324.21,206.6327,96.389,0.7698,...,1.0333,1.76250,1.75,1.55,1771.380000,6077.06,1900.54,2272.54,64.41,2.2425
2020-01,2342.41,2427.69,4332.237,2281.815,324.0671,1589.16,301.55,174.8693,97.390,0.7554,...,1.0380,1.66188,1.75,1.55,1773.090000,6031.21,1923.93,2354.31,63.76,2.0305
